In [5]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import csv
import sys 

sys.path.append("/home/souly/Desktop/ml/ml-project1")
from src import helpers, preprocessing, exploration, polynomial_exp
from src.helpers import split_data_rand
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from src.preprocessing import undefined_to_median, undefined_to_mean, prune_undefined

In [6]:
x_df = pd.read_csv("../data/raw/x_train.csv")
y_df = pd.read_csv("../data/raw/y_train.csv")

In [10]:
# preprocessing
seed = 23
degree = 1

x = x_df.values
y = y_df["_MICHD"].values

positive_indices = np.where(y == 1)[0]
negative_indices = np.where(y != 1)[0]

min_samples = min(len(positive_indices), len(negative_indices))

random.seed(seed)
downsampled_negative_indices = random.sample(list(negative_indices), min_samples)

balanced_indices = np.concatenate([positive_indices, downsampled_negative_indices])
random.shuffle(balanced_indices)

balanced_x = x[balanced_indices]
balanced_y = y[balanced_indices]

split_ratio = 0.8  # 80% train, 20% test
split_index = int(len(balanced_x) * split_ratio)

x_train = balanced_x[:split_index]
y_train = balanced_y[:split_index]
x_test = balanced_x[split_index:]
y_test = balanced_y[split_index:]

x_train = preprocessing.clean_features(x_train)
x_test = preprocessing.clean_features(x_test)

x_train = undefined_to_mean(x_train)
x_train = helpers.standardize(x_train)
print(x_train.shape)

x_test = undefined_to_mean(x_test)
x_test = helpers.standardize(x_test)
print(x_test.shape)

28975
299160
28975
(46360, 27)
(11590, 27)


In [8]:
model = LogisticRegression()
model.fit(x_train, y_train)
print("Accuracy: " + str(model.score(x_test, y_test)))
pred = model.predict(x_test)
print("F1: " + str(f1_score(y_test, pred)))

Accuracy: 0.7262295081967213
F1: 0.7301641296028573


/home/souly/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
